In [ ]:
import numpy as np
import cv2

In [ ]:
# Images loading with the size check 
outer_img = cv2.imread("background1.jpg")
inner_img = cv2.imread("wormhole_view.jpg")

# Size check
if outer_img.shape != inner_img.shape:
    inner_img = cv2.resize(inner_img, (outer_img.shape[1], outer_img.shape[0]))

# Parameters
center = (outer_img.shape[1] // 2, outer_img.shape[0] // 2)

h, w = outer_img.shape[:2]
y, x = np.ogrid[:h, :w]

In [ ]:
# Wormhole region mask (r <= b)
b = 4

strength = 300

mask = (x - center[0])**2 + (y - center[1])**2 <= (b*strength)**2

# Polar coordinates
dx = x - center[0]
dy = y - center[1]
r = np.sqrt(dx**2 + dy**2)
theta = np.arctan2(dy, dx)

In [ ]:
# Radius distortion for the OUTER image 
r_outer = (b*strength) * np.log(r / (b*strength) + np.sqrt((r / (b*strength))**2 - 1 + 1e-10))

# k = b / r_t 
# phi = np.arccos(r_t*strength / r)
# integral = np.zeros_like(r)
# for i in range(len(r[:,0])):
#     for j in range(len(r[0,:])):
#         integral[i,j] = ellipkinc(phi[i,j], k**2)
# r_outer = r_t*strength/k*integral
r_outer = np.nan_to_num(r_outer)

# Radius distortion for the INNER image 
r_inner = np.where(r < b*strength, np.where(r > 5, (b*strength)**2 / r, 0), r)

C:\Users\Alexander\AppData\Local\Temp\ipykernel_14828\1758084928.py:2: RuntimeWarning: invalid value encountered in sqrt
  r_outer = (b*strength) * np.log(r / (b*strength) + np.sqrt((r / (b*strength))**2 - 1 + 1e-10))
C:\Users\Alexander\AppData\Local\Temp\ipykernel_14828\1758084928.py:14: RuntimeWarning: divide by zero encountered in divide
  r_inner = np.where(r < b*strength, np.where(r > 5, (b*strength)**2 / r, 0), r)


In [ ]:
# New coordinates with borders check 
def safe_coords(r_dist, phi):
    x_new = center[0] + r_dist * np.cos(theta)
    y_new = center[1] + r_dist * np.sin(theta)
    x_new = np.clip(x_new, 0, w-1).astype(int)
    y_new = np.clip(y_new, 0, h-1).astype(int)
    return y_new, x_new

y_outer, x_outer = safe_coords(r_outer, theta)
y_inner, x_inner = safe_coords(r_inner, theta)

# Creating a result
result = outer_img.copy()

# Outer part (neat indexing) 
outer_pixels = outer_img[y_outer, x_outer]
result[~mask] = outer_pixels[~mask]

# Inner part (with mask check)
inner_pixels = inner_img[
    np.clip(y_inner[mask], 0, inner_img.shape[0]-1),
    np.clip(x_inner[mask], 0, inner_img.shape[1]-1)
]
result[mask] = inner_pixels

# Saving
cv2.imwrite("figures\\baikal_galaxy_b=4.jpg", result)

True

In [7]:
r_t

482